# Accomodation Cost Analyser

Data has been successfully scraped and exported to 'scraped_data.xlsx'.


In [5]:
import requests

# Send a GET request to the URL
url = "https://www.daft.ie/property-for-rent/dublin-city?sort=publishDateDesc"
response = requests.get(url)

# Print the HTML content
print(response.text)



<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <title>Service Unavailable | Daft</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <style>/* cyrillic-ext */
@font-face {
  font-family: 'Open Sans';
  font-style: normal;
  font-weight: 400;
  font-stretch: normal;
  font-display: swap;
  src: url(data:font/woff2;base64,d09GMgABAAAAADIcAA8AAAAAYdgAADG/AAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGk4biyIcPAZgP1NUQVReAHQREAqBkjT4AQuDOAABNgIkA4ZkBCAFhHgHkCEb5VEV45iluB2AiFJt/ojK1YlEUTY4yeD/Twl0DJHiTAH1FwKCILNroXtmYgSaMFGnO4pbYR2LswmSvtUtb4WEfUZeRn7YxmKWHj8SECMoagvf9E1usNFhha/+KXiTIzT2SS48//n3X/uo7vuvacWiol4dGzMTyYrIGoJtdipYK6fotNGeMzALETEJsVCxwAILMbACAyNrM4cdmwvn1q7y/xf5vTA3MrRO80jUWqh0c51cDOep2tN8N92z+7XCYgxVqAgfFSEdMtzAPHMexeM+p5+zASRmFrgQX3922SB+DthZaZkG3u/A7i7GgKQbmGdfTcuV0IWluoRGyEI5QY9/i+oYoRNCHX64328H7Q1zT18UbhsSVd

In [36]:
import re
import requests
import pandas as pd

class Daft:
    _ENDPOINT = "https://gateway.daft.ie/old/v1/listings"
    _HEADER = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Content-Type": "application/json",
        "brand": "daft",
        "platform": "web",
    }
    _PAGE_SZ = 50
    _PAGE_0 = {"from": "0", "pagesize": str(_PAGE_SZ)}

    def __init__(self):
        self._section = None

    def get_listings(self):
        response = requests.post(
            self._ENDPOINT,
            headers=self._HEADER,
            json=self._PAGE_0
        )
        
        if response.status_code == 200:
            listings = response.json()
            return listings
        else:
            print(f"Failed to retrieve listings. Status code: {response.status_code}")
            return None

# Initialize Daft instance
daft_instance = Daft()

# Get listings
listings_data = daft_instance.get_listings()

if listings_data and 'listings' in listings_data:
    # Extract relevant information from listings_data
    listings = listings_data['listings']
    
    # Create a list to store dictionaries of listing data
    data = []

    # Populate the list
    for listing in listings:
        title = listing['listing']['title']
        location = listing['listing']['seoTitle']
        
        # Extract numeric part from the price text
        price_text = listing['listing']['price']
        price_numeric = int(re.sub(r'[^\d]', '', price_text))
        
        # Extracting maximum number from the bedrooms text using regular expressions
        bedrooms_text = listing['listing']['numBedrooms']
        bedrooms_numeric = max(map(int, re.findall(r'\d+', bedrooms_text)))
        
        data.append({'Title': title, 'Location': location, 'Price': price_numeric, 'Bedrooms': bedrooms_numeric})

    # Create DataFrame from the list
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    df.to_excel('daft_listings.xlsx', index=False)

    print("Data saved to daft_listings.xlsx")
else:
    print("No listings data retrieved")


Data saved to daft_listings.xlsx
